In [3]:
# TODO : generator
from lib.ekyn import get_dataloaders
from sage.utils import *
from sage.models import *
from time import time
import datetime
import copy
import os


resnet_architectures = [ #(widthi,depthi)
    ([4,8],[1,3]),
    ([4,8],[1,4]),
    ([4,8,16],[1,1,1]),
    ([4,8,16],[1,1,2]),
    ([4,8,16],[1,1,4]),
    ([4,8,16],[1,2,1]),
    ([4,8,16],[1,2,2]),
    ([4,8,16],[1,2,4]),
    ([16,32],[1,2]),
    ([32,64],[1,1]),
]

group_id = 'batch'
for widthi,depthi in resnet_architectures:
    hyperparameters = {
        'wd':1e-2,
        'lr':3e-3,
        'batch_size':2048,
        'norm':'batch',
        'stem_kernel_size':3,
        'widthi':widthi,
        'depthi':depthi,
        'patience':100,
        'epochs':500
    }

    trainloader,testloader = get_dataloaders(batch_size=hyperparameters['batch_size'])
    model = ResNetv2(ResBlockv2,widthi=hyperparameters['widthi'],depthi=hyperparameters['depthi'],n_output_neurons=3,norm=hyperparameters['norm'],stem_kernel_size=hyperparameters['stem_kernel_size'])
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(),lr=hyperparameters['lr'],weight_decay=hyperparameters['wd'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)

    state = {
        'start_time':datetime.datetime.now().strftime("%Y_%d_%m_%H_%M_%S"),
        'group_id':group_id,
        'execution_time':0,
        'device':'cuda',
        'trainlossi':[],
        'testlossi':[],
        'best_dev_loss':torch.inf,
        'model':model,
        'scheduler':scheduler,
        'criterion':criterion,
        'optimizer':optimizer,
        'best_model_wts':copy.deepcopy(model.state_dict()),
    }

    os.makedirs(f'experiments/{state["start_time"]}')

    for key in hyperparameters:
        state[key] = hyperparameters[key]

    last_time = time()

    for state in train(state,trainloader,testloader):
        plot_loss(state)
        state['execution_time'] = (state['execution_time'] + (time() - last_time))/2
        last_time = time()
        torch.save(state, f'experiments/{state["start_time"]}/state.pt')
    torch.save(state, f'experiments/{state["start_time"]}/state.pt')


/home/andrew/sleep/sleep_ml/lib/ekyn.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return load(f'{DATA_PATH}/pt_ekyn_robust_50hz/{id}_{condition}.pt')


train_ids ['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1']
test_ids ['A1-1', 'C4-0', 'D1-0', 'E1-0']
n ids 16
102 training batches 34 testing batches
208896 training samples 69632 testing samples
580.27 training hours 193.42 testing hours
1383 params
500


0.5633,0.4586:   0%|          | 1/500 [00:02<22:52,  2.75s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78b532784ca0>
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/home/andrew/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
0.3527,0.3891:   1%|          | 3/500 [00:11<31:29,  3.80s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78b532784ca0>
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()


train_ids ['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1']
test_ids ['A1-1', 'C4-0', 'D1-0', 'E1-0']
n ids 16
102 training batches 34 testing batches
208896 training samples 69632 testing samples
580.27 training hours 193.42 testing hours
1799 params
500


0.2615,0.2647:  38%|███▊      | 192/500 [11:17<18:06,  3.53s/it]
/home/andrew/sleep/sleep_ml/lib/ekyn.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return load(f'{DAT

...

0.2548,0.2694:  33%|███▎      | 166/500 [08:08<16:22,  2.94s/it]
/home/andrew/sleep/sleep_ml/lib/ekyn.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return load(f'{DAT

train_ids ['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1']
test_ids ['A1-1', 'C4-0', 'D1-0', 'E1-0']
n ids 16
102 training batches 34 testing batches
208896 training samples 69632 testing samples
580.27 training hours 193.42 testing hours
7167 params
500


0.2447,0.2628:  25%|██▍       | 123/500 [10:10<31:12,  4.97s/it]  
/home/andrew/sleep/sleep_ml/lib/ekyn.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return load(f'{D

train_ids ['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1']
test_ids ['A1-1', 'C4-0', 'D1-0', 'E1-0']
n ids 16
102 training batches 34 testing batches
208896 training samples 69632 testing samples
580.27 training hours 193.42 testing hours
13651 params
500


0.2243,0.2657:  15%|█▌        | 75/500 [04:18<24:56,  3.52s/it]